<a href="https://colab.research.google.com/github/xoyeon/hongong-mldl/blob/main/07_2_%EC%8B%AC%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

인공 신경망에 층을 여러 개 추가하여 패션 MNIST 데이터셋을 분류하면서 케라스로 심층 신경망을 만드는 방법을 자세히 배웁니다.

# 2개의 층

In [1]:
# 케라스 API를 사용해서 패션 MNIST 데이터셋 불러오기
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [2]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

은닉층(hidden layer) : 입력층과 출력층 사이에 있는 모든 층   

은닉층에는 활성화 함수가 있음
- 활성화 함수는 신경망 층의 선형 방적식의 계산 값에 적용하는 함수
- 이진 분류: 시그모이드, 다중 분류: 소프트맥스

In [8]:
# 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어보자
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) ## 은닉층, 100개의 뉴런을 가진 밀집층
dense2 = keras.layers.Dense(10, activation='softmax') ## 출력층

# 심층 신경망 만들기

In [9]:
model = keras.Sequential([dense1, dense2]) ## 출력층을 가장 마지막에 두어야 함

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


#층을 추가하는 다른 방법

In [11]:
# 방법2: Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 多
model = keras.Sequential([
                          keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),name='hidden'),
                          keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [12]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 방법3: Sequential 클래스의 객체를 만들고 이 객체의 add() 메서드를 호출하여 층을 추가
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5603 - accuracy: 0.8093
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4072 - accuracy: 0.8543
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3732 - accuracy: 0.8654
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3512 - accuracy: 0.8721
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3322 - accuracy: 0.8793


# 렐루 함수
입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만들며, 심층 신경망에서 뛰어남

In [18]:
# Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할. 입력층 바로 뒤에 추가.
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


케라스 API는 입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시켜야 함

In [20]:
# 모델 훈련
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.1, random_state=42
)

In [21]:
# 모델 컴파일 후 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5254 - accuracy: 0.8149
Epoch 2/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3909 - accuracy: 0.8605
Epoch 3/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3542 - accuracy: 0.8732
Epoch 4/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3328 - accuracy: 0.8817
Epoch 5/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3186 - accuracy: 0.8881


In [22]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

188/188 [==============================] - 0s 1ms/step - loss: 0.3967 - accuracy: 0.8650


[0.3966924250125885, 0.8650000095367432]

# 옵티마이저
- 확률적 경사 하강법인 SGD
> momentum 매개변수의 기본값은 0   
> 0보다 큰 값으로 지정하면 모멘텀 최적화(momentum optimization)사용. 매개변수는 0.9 이상

- SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 네스테로프 모멘텀 최적화(nesterov momentum optimizaton)사용   
: 최적화를 2번 반복하여 구현  -> 이런 학습률을 적응적 학습률(adaptive learning rate)
> Adagrad, RMSporp

In [24]:
# Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델 훈련
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [25]:
# compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크 동안 훈련
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5087 - accuracy: 0.8227
Epoch 2/5
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3824 - accuracy: 0.8631
Epoch 3/5
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3442 - accuracy: 0.8747
Epoch 4/5
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3207 - accuracy: 0.8825
Epoch 5/5
1688/1688 [==============================] - 3s 2ms/step - loss: 0.3021 - accuracy: 0.8895


In [26]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

188/188 [==============================] - 0s 1ms/step - loss: 0.3405 - accuracy: 0.8765


[0.34047800302505493, 0.8765000104904175]